In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from EDA import EDA

## Stratified Sampling, don't run this is AWS

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession \
    .builder \
    .appName("process_data") \
    .getOrCreate()

data_dir = "../MLE_Project_Data"
data_file_name = "train_data.csv"
data_path = os.path.join(data_dir,data_file_name)

raw_df = spark.read.load(data_path,format="csv",delimiter="|",header=True)

# sample only 120,000 ish data for 
sampled_df = raw_df.stat.sampleBy("label",
                     {"0":0.01,"1":0.01},
                     seed=100)

# save data, don't
sample_file_name = "sampled_data.csv"
ouptput_dir = "./data"
sampled_df.write.format('csv').option('header',True)\
.mode('overwrite').option('sep',',')\
.save(os.path.join(ouptput_dir,sample_file_name))